In [1]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [15]:
import sys
import re
import json

from nltk.corpus import stopwords

path_json = "music2.json"

with open(path_json) as data_file:    
    data = json.load(data_file)

sentences = []
names = []
reviews = []

for key in data:
    tmp =  data[key]
    if "id3" in tmp:
        id = tmp["id3"]
        if ("artist" in id) and ("title" in id):
            artist_name =  id["artist"]
            title_name = id["title"]

            nm = artist_name + ':' + title_name
            lyr = id["lyrics"]

            if (len(lyr)>1):
            
                lyr = re.sub("[^a-zA-Z]"," ", lyr)
                words = lyr.lower().split()

                stops = set(stopwords.words("english"))
                review = [w for w in words if not w in stops]
                
                sentences.append(words)
                names.append(nm)
                reviews.append(review)

In [21]:
model = gensim.models.Word2Vec(sentences, min_count=5, size=200)

In [19]:
names[0]



'Judas Priest:Living Bad Dreams [bonus]'

In [24]:
# from https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-3-more-fun-with-word-vectors


import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       
       # Print a status message every 1000th review
       # if (counter%1000. == 0.):
       #    print ("Review %d of %d", (counter, len(reviews)))
        
       # Call the function (defined above) that makes average feature vectors
    
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [25]:
DataVecs = getAvgFeatureVecs(reviews, model, 200)

In [32]:
DataVecs.shape

(312, 200)

In [33]:
DataVecs

array([[-0.13195308, -0.12742832, -0.16498871, ...,  0.08429646,
        -0.06533853, -0.12015924],
       [-0.18357311, -0.17900413, -0.23386782, ...,  0.10447213,
        -0.08501918, -0.17449413],
       [-0.08983714, -0.08939039, -0.11942608, ...,  0.05918592,
        -0.04546231, -0.08801154],
       ..., 
       [-0.12924011, -0.12829229, -0.16764517, ...,  0.08652055,
        -0.06568135, -0.1228853 ],
       [-0.15096208, -0.14797041, -0.19576186, ...,  0.09285876,
        -0.07299571, -0.14197956],
       [-0.12736443, -0.1237139 , -0.16448508, ...,  0.07961028,
        -0.06193249, -0.11960296]], dtype=float32)

In [55]:
from scipy import spatial

l = DataVecs.shape[0]

In [49]:
#most similar

for i in range(l):
    m = 0
    n = 0
    id = 0
    for j in range(l):
        if (names[i] != names[j]):
            res = 1 - spatial.distance.cosine(DataVecs[i], DataVecs[j])
            if (res > m):
                m = res
                n = i
                id = j
    print(m, n, id, names[n], names[id])

0.999883965286 0 227 Judas Priest:Living Bad Dreams [bonus] Pink Floyd:Sheep
0.999889339551 1 259 The White Stripes:In the Cold, Cold Night Red Hot Chili Peppers:Save the Population
0.999974427056 2 94 Coldplay:X&Y Alice in Chains:Love, Hate, Love
0.999925891489 3 217 Jack White:Missing Pieces Alice in Chains:What the Hell Have I
0.999967140212 4 25 Green Day:Extraordinary Girl Green Day:St. Jimmy
0.999902039921 5 67 Alice in Chains:Over Now Queens of the Stone Age:Avon
0.999927269647 6 237 Foo Fighters:The Pretender Limp Bizkit:My Way
0.999917485153 7 241 Godsmack:Vampires Queens of the Stone Age:My God Is the Sun
0.999900509546 8 40 Coldplay:Don't Panic Red Hot Chili Peppers:Under the Bridge
0.999956282663 9 239 Jack White:Blunderbuss Alice in Chains:Sea of Sorrow
0.999923292824 10 307 Them Crooked Vultures:Warsaw or the First Breath You Take After You Give Up Ozzy Osbourne:Desire
0.999926746598 11 32 Alice in Chains:Dam That River Godsmack:Spiral
0.999977147772 12 86 Jack White:I Gu